# 🚦 Snowflake Trail - Step 2: Observability Setup

---
* Event Table setup
* Log Level setting
* Notification integration for Slack / Teams webhook
* Function to get new event logs as json
* Function to format json for Slack / Teams channels
* Alert on new telemetry events
* Alert cost monitoring

## 2.1. Logging Setup

In [ ]:
-- schema to contain all objects for observability layer

create schema if not exists SNOWTRAIL_DEMO.OBSERV
    comment = 'contains all objects for observability layer';

Define a dedicated Event Table for the selected Database to separate event logs from the rest of the account

See https://docs.snowflake.com/en/developer-guide/logging-tracing/event-table-setting-up#associate-an-event-table-with-an-object

In [ ]:
create event table if not exists SNOWTRAIL_DEMO.OBSERVE.EVENTS;

In [ ]:
alter database SNOWTRAIL_DEMO 
    set EVENT_TABLE = SNOWTRAIL_DEMO.OBSERV.EVENTS;

-- alter database SNOWTRAIL_DEMO 
--     unset EVENT_TABLE;

In [ ]:
-- test query event table in database

select
    to_char(CONVERT_TIMEZONE('UTC','Europe/Berlin', TIMESTAMP), 'YYYY-MM-DD at HH:MI:SS') as LOCAL_TIME,    -- adjust to local timezone
    case when RECORD['severity_text']::string = 'DEBUG'  then '🏗️ DEBUG'
         when RECORD['severity_text']::string = 'INFO'  then 'ℹ️ INFO'
         when RECORD['severity_text']::string = 'WARN'  then '⚠️ WARN'
         when RECORD['severity_text']::string = 'ERROR' then '⛔️ ERROR'
         when RECORD['severity_text']::string = 'FATAL' then '🚨 FATAL'
    end as SEVERITY,       
    upper(try_parse_json(VALUE):state ::string) as EXECUTION_STATUS,
    coalesce(
        try_parse_json(VALUE):message ::string, 
        try_parse_json(VALUE) :first_error_message ::string,                        -- temporary special handling of Pipe logs
        (case
            when position('{"' in VALUE) > 0 
            then left(VALUE, position('{"' in VALUE) - 1)
            else VALUE
            end) ::string
        ) as MESSAGE,
        
    coalesce(
        RESOURCE_ATTRIBUTES['snow.executable.name']::string, 
        RESOURCE_ATTRIBUTES['snow.pipe.name']::string                               -- temporary special handling of Pipe logs
            ) as OBJECT_NAME,
    coalesce(
        RESOURCE_ATTRIBUTES['snow.executable.type']::string, 
        case when RESOURCE_ATTRIBUTES['snow.pipe.name'] is not NULL then 'PIPE' end  -- temporary special handling of Pipe logs
            ) as OBJECT_TYPE,
    RESOURCE_ATTRIBUTES['snow.schema.name']::string as SCHEMA_NAME,
    RESOURCE_ATTRIBUTES['snow.database.name']::string as DATABASE_NAME
from 
    SNOWTRAIL_DEMO.OBSERV.EVENTS                                --adjust to active event table
where 
    RECORD_TYPE in ('LOG', 'EVENT')
    and upper(RESOURCE_ATTRIBUTES['snow.database.name']::string) = 'SNOWTRAIL_DEMO' 
order by
    TIMESTAMP desc
limit 
    100
;

## 2.2. Load new events into a json string

A helper function to query all new event logs from the event table, add some formatting and append them to a string

In [ ]:
create or replace function SNOWTRAIL_DEMO.OBSERV.GET_NEW_EVENTS_AS_JSON(START_TIME timestamp)
returns string
language SQL
comment = 'limited to the latest 10 events'
as
$$
(
    with
    EVENT_COUNTS as(
        select 
            count(*) as EVENTS,
            case 
                when RECORD['severity_text']::string = 'INFO'  then 'ℹ️ INFO'
                when RECORD['severity_text']::string = 'WARN'  then '⚠️ WARN'
                when RECORD['severity_text']::string = 'ERROR' then '⛔️ ERROR'
                when RECORD['severity_text']::string = 'FATAL' then '🚨 FATAL'
                when RECORD['severity_text']::string = 'DEBUG' then '🛠️ DEBUG'
                    end as SEVERITY,
        from 
            SNOWTRAIL_DEMO.OBSERV.EVENTS 
        where 
            TIMESTAMP >= START_TIME
            and RECORD_TYPE in ('LOG', 'EVENT')
        group by
            SEVERITY
        ),
        
    LAST_10_EVENTS as (
        select
            to_char(CONVERT_TIMEZONE('UTC','Europe/Berlin', TIMESTAMP), 'YYYY-MM-DD at HH:MI:SS')   -- adjust to local timezone
                as LOCAL_TIME,    
            case 
                when RECORD['severity_text']::string = 'INFO'  then 'ℹ️ INFO'
                when RECORD['severity_text']::string = 'WARN'  then '⚠️ WARN'
                when RECORD['severity_text']::string = 'ERROR' then '⛔️ ERROR'
                when RECORD['severity_text']::string = 'FATAL' then '🚨 FATAL'
                when RECORD['severity_text']::string = 'DEBUG' then '🛠️ DEBUG'
                    end as SEVERITY,
                    
            upper(try_parse_json(VALUE):state ::string) 
                as EXECUTION_STATUS,
                
            coalesce(
                split_part(RESOURCE_ATTRIBUTES['snow.executable.name'],':',0) ::string,     -- trimming procedure arguments 
                RESOURCE_ATTRIBUTES['snow.pipe.name']::string                               -- temporary special handling of Pipe logs
                    ) as OBJECT_NAME,
                    
            coalesce(
                replace(RESOURCE_ATTRIBUTES['snow.executable.type'],'_','-')::string, 
                case when RESOURCE_ATTRIBUTES['snow.pipe.name'] is not NULL then 'PIPE' end  -- temporary special handling of Pipe logs
                    ) as OBJECT_TYPE,
                    
            VALUE['state']::string 
                    as OBJECT_STATE,
                    
            coalesce(
                try_parse_json(VALUE):message ::string, 
                try_parse_json(VALUE) :first_error_message ::string,                        -- temporary special handling of Pipe logs
                (case
                    when position('{"' in VALUE) > 0 
                    then left(VALUE, position('{"' in VALUE) - 1)
                    else VALUE                                                              -- catching messages from custom logs
                    end) ::string
                ) as MESSAGE,
                
            RESOURCE_ATTRIBUTES['snow.schema.name']::string as SCHEMA_NAME,
            RESOURCE_ATTRIBUTES['snow.database.name']::string as DATABASE_NAME,
            current_account_name() ::string as ACCOUNT_NAME,
            
            'https://app.snowflake.com/'||lower(CURRENT_ORGANIZATION_NAME())||'/'|| lower(CURRENT_ACCOUNT_NAME()) ||'/#/data/databases/'|| DATABASE_NAME ||'/schemas/'|| SCHEMA_NAME ||'/'||lower(OBJECT_TYPE)||'/'||OBJECT_NAME 
                as OBJECT_URL,
                
            'https://app.snowflake.com/'||lower(CURRENT_ORGANIZATION_NAME())||'/'||lower(CURRENT_ACCOUNT_NAME()) ||'/#/compute/history/queries/'|| RESOURCE_ATTRIBUTES['snow.query.id']::string ||'/telemetry' 
                as QUERY_URL
                
        from 
            SNOWTRAIL_DEMO.OBSERV.EVENTS                                                    --adjust to active event table
        where 
            TIMESTAMP >= START_TIME
            and RECORD_TYPE in ('LOG', 'EVENT')
        order by
            TIMESTAMP desc
        limit 
            10
        )
        
    select
        OBJECT_CONSTRUCT(
            'count_new_events', (select 
                ARRAY_AGG(OBJECT_CONSTRUCT(
                    'severity', SEVERITY,
                    'events', EVENTS
                ))
                from
                    EVENT_COUNTS
                ),
            'recent_events', (select
                ARRAY_AGG(OBJECT_CONSTRUCT(
                   'local_time', LOCAL_TIME,
                   'severity', SEVERITY,
                   'object_name', OBJECT_NAME,
                   'object_type', OBJECT_TYPE,
                   'object_state', OBJECT_STATE,
                   'message', MESSAGE,
                   'schema', SCHEMA_NAME,
                   'database', DATABASE_NAME,
                   'account', ACCOUNT_NAME,
                   'object_url', OBJECT_URL,
                   'query_url', QUERY_URL
                ))
                from
                    LAST_10_EVENTS
                )
            )
        )::string
$$
;

In [ ]:
-- we can test our new UDF

select SNOWTRAIL_DEMO.OBSERV.GET_NEW_EVENTS_AS_JSON(
    timeadd(hour, -1, current_timestamp)
);

---

## 2.3. Notification setup

Below are 4 alternative options to set up notifications:

A) Slack

B) Microsoft Teams

C) Amazon SNS Topic

D) E-mail

### Option A) Slack message

In [ ]:
# run this cell to show the temporary input field for your Webhook  

import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()

st.divider()
col1, col2 = st.columns([1,1])
col1.caption('Enter the webhook for the Slack channel you want to connect to. Only the part after https://hooks.slack.com/services/')
MY_SLACK_WEBHOOK = col1.text_input("Webhook")
if MY_SLACK_WEBHOOK == "":
    raise Exception("Teams channel webhook needed to create notification integration")

In [ ]:
--- get the secret from your Slack channel, see Slack documentation for details

create or replace secret SNOWTRAIL_DEMO.OBSERV.DEMO_SLACK_WEBHOOK
    type = GENERIC_STRING
    secret_string = '{{MY_SLACK_WEBHOOK}}'
;

In [ ]:
-- see https://docs.snowflake.com/sql-reference/sql/create-notification-integration-webhooks

create or replace notification integration SNOWTRAIL_DEMO_SLACK_CHANNEL
    type = WEBHOOK
    enabled = TRUE
    webhook_url = 'https://hooks.slack.com/services/SNOWFLAKE_WEBHOOK_SECRET'
    webhook_secret = SNOWTRAIL_DEMO.OBSERV.DEMO_SLACK_WEBHOOK
    webhook_headers = ('Content-Type'='text/json')
    comment = 'posting to Channel in Slack workspace'
;

In [ ]:
call SYSTEM$SEND_SNOWFLAKE_NOTIFICATION(
  SNOWFLAKE.NOTIFICATION.APPLICATION_JSON('{"text": "Hello from Snowflake"}'),
  SNOWFLAKE.NOTIFICATION.INTEGRATION('SNOWTRAIL_DEMO_SLACK_CHANNEL')
);

In [ ]:
--- new dynamic function that converts event logs into json blocks for slack message

create or replace function SNOWTRAIL_DEMO.OBSERV.SLACK_MESSAGE_FROM_JSON("EVENT_LOGS" VARCHAR)
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'GENERATE_JSON_BLOCKS_FOR_SLACK'
as $$

import json

def GENERATE_JSON_BLOCKS_FOR_SLACK(EVENT_LOGS):

    try:
        EVENT_DATA = json.loads(EVENT_LOGS)
    except Exception as e:
        return json.dumps({"error": "Invalid JSON input", "details": str(e)})

    SEVERITY_COUNTS = EVENT_DATA.get("count_new_events", [])
    EVENTS = EVENT_DATA.get("recent_events", [])

    TOTAL_EVENTS = sum(item.get("events", 0) for item in SEVERITY_COUNTS)
    
    BLOCKS = []


    
# adding total count

    HEADER_BLOCK = {
        "type": "header",
        "text": {
            "type": "plain_text",
            "text": f"{TOTAL_EVENTS} new events since last check"
            }
        }
    BLOCKS.append(HEADER_BLOCK)



# adding count by severity    

    SEVERITY_ORDER = ['🚨 FATAL', '⛔️ ERROR', '⚠️ WARN', 'ℹ️ INFO', '🛠️ DEBUG']
    SEVERITY_MAP = {item.get("severity", "").upper(): item.get("events", 0) for item in SEVERITY_COUNTS}

    SEV_TEXT_PARTS = []
    for SEV_TYPE in SEVERITY_ORDER:
        COUNT = SEVERITY_MAP.get(SEV_TYPE.upper(), 0)
        if COUNT:
            SEV_TEXT_PARTS.append(f"{SEV_TYPE}: {COUNT}")

    ALL_SEVERITY_COUNTERS = " | ".join(SEV_TEXT_PARTS)

    COUNTER_BLOCK = {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f"```{ALL_SEVERITY_COUNTERS}```"
        }
    }
    BLOCKS.append(COUNTER_BLOCK)


    
# adding the first 10 events  

    for EVENT in EVENTS[:10]:

        BLOCKS.append({"type": "divider"})
        
        fields = []
        
    # Account
        account = EVENT.get('account', 'N/A')
        fields.append({
            "type": "mrkdwn",
            "text": f"Account:\n *{account}*"
        })
        
    # Severity
        severity = EVENT.get('severity', '')
        fields.append({
            "type": "mrkdwn",
            "text": f"Severity:\n *{severity}*"
        })
        
    # Database and schema
        database = EVENT.get('database', '')
        schema = EVENT.get('schema', '')
        fields.append({
            "type": "mrkdwn",
            "text": f"Schema:\n *{database}.{schema}*"
        })
        
    # Local time
        local_time = EVENT.get('local_time', '')
        fields.append({
            "type": "mrkdwn",
            "text": f"Local time:\n `{local_time}`"
        })
        
    # Object_type and object_name
        object_name = EVENT.get('object_name', '')
        object_type = EVENT.get('object_type', '')
        object_type_formatted = object_type[0].upper() + object_type[1:].lower() if object_type else " "
        fields.append({
            "type": "mrkdwn",
            "text": f"Object:\n *{object_type_formatted} {object_name}*"
        })
        
    # Object status
        object_state = EVENT.get('object_state', '')
        if object_state:
            fields.append({
                "type": "mrkdwn",
                "text": f"Object status:\n `{object_state}`"
            })
            
        
        section_fields_block = {
            "type": "section",
            "fields": fields
        }
        BLOCKS.append(section_fields_block)

        
    # Error message
        error_message = EVENT.get("message", " ")
        error_section_block = {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"```{error_message}```"
            }
        }
        if error_message:
            BLOCKS.append(error_section_block)

       
        BUTTONS = []

    # Go to object button
        object_url = EVENT.get("object_url")
        if object_url:
            BUTTONS.append({
                "type": "button",
                "text": {
                    "type": "plain_text",
                    "text": "Go to Object"
                },
                "style": "primary",
                "value": "Go to Object",
                "url": object_url
            })

    # Go to query button
        event_url = EVENT.get("query_url")
        if event_url:
            BUTTONS.append({
                "type": "button",
                "text": {
                    "type": "plain_text",
                    "text": "Go to Event"
                },
                "value": "Go to Event",
                "url": event_url
            })
        
        if BUTTONS:
            ACTION_BLOCK = {
                "type": "actions",
                "elements": BUTTONS
            }
            BLOCKS.append(ACTION_BLOCK)
            
    return json.dumps({"blocks": BLOCKS})

$$
;

In [ ]:
--- testing our 2 UDFs inside the system function to send notifications

call SYSTEM$SEND_SNOWFLAKE_NOTIFICATION(                                    -- send notification (system function)
        SNOWFLAKE.NOTIFICATION.APPLICATION_JSON(                            -- in json format (system function)
            SNOWTRAIL_DEMO.OBSERV.SLACK_MESSAGE_FROM_JSON(                  -- json formatted as Slack blocks (UDF)
                SNOWTRAIL_DEMO.OBSERV.GET_NEW_EVENTS_AS_JSON(               -- get all new event logs as a json string (UDF)
                    timeadd(hour, -1, current_timestamp)        
                )
            )
        ),
        SNOWFLAKE.NOTIFICATION.INTEGRATION('SNOWTRAIL_DEMO_SLACK_CHANNEL')  -- using this notification integration
    )
;

In [ ]:
create or replace alert SNOWTRAIL_DEMO.OBSERV.NEW_ERRORS
-- no warehouse -> serverless
-- no schedule -> triggered by new data
comment = 'Streaming Alert on Event Table to notify in Slack asap'
if(exists(
    select 
        * 
    from 
        SNOWTRAIL_DEMO.OBSERV.EVENTS 
    where
        RECORD_TYPE in ('EVENT', 'LOG')
        and upper(RESOURCE_ATTRIBUTES:"snow.schema.name") = 'PIPELINE'              -- optional
        -- and upper(RESOURCE_ATTRIBUTES:"snow.database.name") = 'SNOWTRAIL_DEMO'   -- not needed as scope is this database only
        ))
then
    begin
        let FIRST_NEW_TIMESTAMP timestamp :=(
            select min(TIMESTAMP) from table(RESULT_SCAN(SNOWFLAKE.ALERT.GET_CONDITION_QUERY_UUID()))      -- get query ID from condition query above
        );
    
        call SYSTEM$SEND_SNOWFLAKE_NOTIFICATION(                                -- send notification (system function)
            SNOWFLAKE.NOTIFICATION.APPLICATION_JSON(                            -- in json format (system function)
                SNOWTRAIL_DEMO.OBSERV.SLACK_MESSAGE_FROM_JSON(                  -- json formatted as Slack blocks (UDF)                 
                    SNOWTRAIL_DEMO.OBSERV.GET_NEW_EVENTS_AS_JSON(               -- get all new event logs as a json string (UDF)
                        :FIRST_NEW_TIMESTAMP
                    )
                )
            ),
            SNOWFLAKE.NOTIFICATION.INTEGRATION('SNOWTRAIL_DEMO_SLACK_CHANNEL')  -- using this notification integration
        );
    end;
;

### Option B) Microsoft Teams message

In [ ]:
# run this cell to show the temporary input field for your Webhook  

import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()

st.divider()
col1, col2 = st.columns([1,1])
col1.caption('Enter the webhook for the Teams channel you want to connect to. Only the part after https://mymsofficehost.webhook.office.com/webhookb2/')
MY_TEAMS_WEBHOOK = col1.text_input("Webhook")
MY_MS_OFFICE_HOST = col1.text_input("MS Office Host")
if MY_TEAMS_WEBHOOK == "":
    raise Exception("Teams channel URL needed to create notification integration")
if MY_MS_OFFICE_HOST == "":
    raise Exception("company domain for MS office needed to create notification integration")

In [ ]:
create or replace secret SNOWTRAIL_DEMO.OBSERV.SNOWTRAIL_DEMO_TEAMS_WEBHOOK
    type = GENERIC_STRING
    secret_string = '{{MY_TEAMS_WEBHOOK}}'
;

In [ ]:
-- see https://docs.snowflake.com/sql-reference/sql/create-notification-integration-webhooks

create or replace notification integration SNOWTRAIL_DEMO_TEAMS_CHANNEL
    type = WEBHOOK
    enabled = TRUE
    webhook_url = 'https://{{MY_MS_OFFICE_HOST}}.webhook.office.com/webhookb2/SNOWFLAKE_WEBHOOK_SECRET'
    webhook_secret = SNOWTRAIL_DEMO.OBSERV.SNOWTRAIL_DEMO_TEAMS_WEBHOOK
    webhook_body_template=$${
          "type": "message",
          "attachments": [
                {
                  "contentType": "application/vnd.microsoft.card.adaptive",
                  "content": SNOWFLAKE_WEBHOOK_MESSAGE
                }
              ]
            }$$
    webhook_headers = ('Content-Type'='application/json')
    comment = 'sending Snowflake notifications to Teams channel'
;

In [ ]:
--- new dynamic function that converts event logs into json blocks for Microsoft Teams message

create or replace function SNOWTRAIL_DEMO.OBSERV.TEAMS_MESSAGE_FROM_JSON("EVENT_LOGS" VARCHAR)
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'GENERATE_ADAPTIVE_CARD_FOR_TEAMS'
as $$
import json

def GENERATE_ADAPTIVE_CARD_FOR_TEAMS(EVENT_LOGS):

    try:
        data = json.loads(EVENT_LOGS)
    except Exception as e:
        error_card = {
            "$schema": "https://adaptivecards.io/schemas/adaptive-card.json",
            "type": "AdaptiveCard",
            "version": "1.5",
            "body": [
                {"type": "TextBlock", "weight": "Bolder", "text": "Snowflake Alert Error", "color": "Attention", "wrap": True, "style": "heading"},
                {"type": "TextBlock", "text": f"Invalid JSON input: {str(e)}", "wrap": True}
            ],
            "msTeams": {"width": "full"}
        }
        return json.dumps(error_card)

        
    severity_counts = data.get("count_new_events", [])
    recent_events = data.get("recent_events", [])

    # Define severity order and style mappings
    SEVERITY_ORDER = ['🚨 FATAL', '⛔️ ERROR', '⚠️ WARN', 'ℹ️ INFO', '🛠️ DEBUG']
    style_map = {
        '🚨 FATAL': 'attention',
        '⛔️ ERROR': 'attention',
        '⚠️ WARN': 'warning',
        'ℹ️ INFO': 'accent',
        '🛠️ DEBUG': 'emphasis'
    }

    # Total and ordered summary line
    total_events = sum(item.get("events", 0) for item in severity_counts)
    summary_parts = []
    for sev in SEVERITY_ORDER:
        count = next((item.get('events', 0) for item in severity_counts if item.get('severity') == sev), 0)
        if count:
            summary_parts.append(f"{sev}: {count}")
    summary_line = " | ".join(summary_parts)

    # Build root card
    card = {
        "$schema": "https://adaptivecards.io/schemas/adaptive-card.json",
        "type": "AdaptiveCard",
        "version": "1.5",
        "msteams": {"width": "full"},
        "body": []
    }

    # Header
    card['body'].append({
        "type": "TextBlock",
        "weight": "Bolder",
        "text": f"{total_events} new Snowflake Events logged",
        "wrap": True,
        "style": "heading"
    })

    # Single-line severity summary
    card['body'].append({
        "type": "TextBlock",
        "text": summary_line,
        "wrap": True,
        "fontType": "Monospace",
        "weight": "Bolder",
        "separator": True,
        "horizontalAlignment": "Left"
    })

    # Event details for the first 10 events
    for event in recent_events[:10]:
        sev = event.get('severity', 'ℹ️ INFO')
        container_style = style_map.get(sev, 'emphasis')

        db = event.get('database', '')
        schema_name = event.get('schema', '')
        obj_name = event.get('object_name', '')
        obj_type = event.get('object_type', '')
        obj_fmt = f"{obj_type.capitalize()} {obj_name}".strip()

        col1 = [
            {"type": "TextBlock", "text": f"**Account:** {event.get('account', 'N/A')}", "wrap": True},
            {"type": "TextBlock", "text": f"**Schema:** {db}.{schema_name}", "wrap": True},
            {"type": "TextBlock", "text": f"**Object:** {obj_fmt}", "wrap": True}
        ]
        col2 = [
            {"type": "TextBlock", "text": f"**Severity:** {sev}", "wrap": True},
            {"type": "TextBlock", "text": f"**Local time:** {event.get('local_time', '')}", "wrap": True}
        ]

        container = {
            "type": "Container",
            "items": [
                {"type": "TextBlock", "size": "Medium", "weight": "Bolder", "text": obj_name or "Event Detail", "wrap": True},
                {"type": "ColumnSet", "columns": [
                    {"type": "Column", "width": 50, "items": col1},
                    {"type": "Column", "width": 50, "items": col2}
                ]}
            ],
            "style": container_style,
            "bleed": True,
            "separator": True
        }

        if event.get('message'):
            container['items'].append({
                "type": "TextBlock",
                "text": f"```{event.get('message')}```",
                "wrap": True,
                "fontType": "Monospace"
            })

        actions = []
        if event.get('object_url'):
            actions.append({"type": "Action.OpenUrl", "title": "Go to Object", "url": event['object_url']})
        if event.get('query_url'):
            actions.append({"type": "Action.OpenUrl", "title": "Go to Query", "url": event['query_url']})
        if actions:
            container['items'].append({"type": "ActionSet", "actions": actions})

        card['body'].append(container)

    return json.dumps(card, ensure_ascii=False)
$$;


In [ ]:
--- testing our 2 UDFs inside the system function to send notifications

select 
    SNOWTRAIL_DEMO.OBSERV.TEAMS_MESSAGE_FROM_JSON(                  -- json formatted as Teams Cards (UDF)
        SNOWTRAIL_DEMO.OBSERV.GET_NEW_EVENTS_AS_JSON(               -- get all new event logs as a json string (UDF)
            timeadd(hour, -1, current_timestamp)        
        )
    )   
;

In [ ]:
create or replace alert SNOWTRAIL_DEMO.OBSERV.NEW_ERRORS
-- no warehouse -> serverless
-- no schedule -> triggered by new data
comment = 'Streaming Alert on Event Table to notify in Teams channel'
if(exists(
    select 
        * 
    from 
        SNOWTRAIL_DEMO.OBSERV.EVENTS 
    where
        RECORD_TYPE in ('EVENT', 'LOG')
        and upper(RESOURCE_ATTRIBUTES:"snow.schema.name") = 'PIPELINE'              -- optional
        -- and upper(RESOURCE_ATTRIBUTES:"snow.database.name") = 'SNOWTRAIL_DEMO'   -- not needed as scope is this database only
        ))
then
    begin
        let FIRST_NEW_TIMESTAMP timestamp :=(
            select min(TIMESTAMP) from table(RESULT_SCAN(SNOWFLAKE.ALERT.GET_CONDITION_QUERY_UUID()))      -- get query ID from condition query above
        );
    
        call SYSTEM$SEND_SNOWFLAKE_NOTIFICATION(                                -- send notification (system function)
            SNOWFLAKE.NOTIFICATION.APPLICATION_JSON(                            -- in json format (system function)
                SNOWTRAIL_DEMO.OBSERV.TEAMS_MESSAGE_FROM_JSON(                  -- json formatted as Teams cards (UDF)                 
                    SNOWTRAIL_DEMO.OBSERV.GET_NEW_EVENTS_AS_JSON(               -- get all new event logs as a json string (UDF)
                        :FIRST_NEW_TIMESTAMP
                    )
                )
            ),
            SNOWFLAKE.NOTIFICATION.INTEGRATION('SNOWTRAIL_DEMO_TEAMS_CHANNEL')  -- using this notification integration
        );
    end;
;

### Option C) Amazon SNS message

⚠️ Currently, this feature is limited to Snowflake accounts hosted on AWS.

Sample output format:

```  
{
  "summary": "🚨 FATAL: 2 | ⛔️ ERROR: 1 | ...",
  "recent_events": [
    {
      "local_time": "2025-04-16 at 10:39:22",
      "severity": "🚨 FATAL",
      "account": "DEMO",
      "schema": "PIPELINE",
      "object_type": "FUNCTION",
      "object_name": "MY_FUNCTION",
      "object_status": "",
      "message": "exception",
      "object_url": "https://...",
      "query_url": "https://..."
    },
    ...
  ]
}

```

In [ ]:
# run this cell to show the temporary input fields for your ARNs  

import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()

st.divider()
col1, col2 = st.columns([1,1])
col1.caption('Enter the topic ARN and role ARN for the SNS channel you want to connect to.')
MY_SNS_TOPIC_ARN = col1.text_input("SNS TOPIC ARN")
MY_SNS_ROLE_ARN = col1.text_input("SNS ROLE ARN (case-sensitive)")

if MY_SNS_TOPIC_ARN == "":
    raise Exception("SNS Topic ARN needed to create notification integration")
if MY_SNS_ROLE_ARN == "":
    raise Exception("SNS Role ARN needed to create notification integration")

In [ ]:
-- see https://docs.snowflake.com/en/user-guide/notifications/creating-notification-integration-amazon-sns

create or replace notification integration SNOWTRAIL_DEMO_SNS_TOPIC
    enabled = TRUE
    type = QUEUE
    direction = OUTBOUND
    notification_provider = AWS_SNS
    aws_sns_topic_arn = '{{MY_SNS_TOPIC_ARN}}'
    aws_sns_role_arn = '{{MY_SNS_ROLE_ARN}}'
    comment = 'sending Snowflake notifications to SNS topic'
;

In [ ]:
describe notification integration SNOWTRAIL_DEMO_SNS_TOPIC;

-- Record the values of the following properties:
-- SF_AWS_IAM_USER_ARN
-- SF_AWS_EXTERNAL_ID
-- and add them to your SNS & IAM policies

In [ ]:
--- new dynamic function that converts event logs into json blocks for Amazon SNS

create or replace function SNOWTRAIL_DEMO.OBSERV.SNS_MESSAGE_FROM_JSON("EVENT_LOGS" VARCHAR)
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'GENERATE_JSON_BLOCKS_FOR_SNS'
as
$$
import json

def GENERATE_JSON_BLOCKS_FOR_SNS(EVENT_LOGS):
    try:
        DATA = json.loads(EVENT_LOGS)
    except Exception as e:
        return json.dumps({"error": "Invalid input", "details": str(e)})
    
    SEVERITY_COUNTS = DATA.get("count_new_events", [])
    RECENT_EVENTS = DATA.get("recent_events", [])[:10]

    
    # summary string
    parts = []
    for sev in SEVERITY_COUNTS:
        sev_type = sev.get("severity", "").strip()
        count = sev.get("events", 0)
        if sev_type and count:
            parts.append(f"{sev_type}: {count}")
    ALL_SEVERITY_COUNTERS = " | ".join(parts)

    
    # recent events
    BLOCKS = []
    for event in RECENT_EVENTS:
        EVENT_BLOCK = {
            "local_time": event.get("local_time", ""),
            "severity": event.get("severity", ""),
            "account": event.get("account", ""),
            "schema": event.get("schema", ""),
            "object_type": event.get("object_type", ""),
            "object_name": event.get("object_name", ""),
            "object_status": event.get("object_state", ""),
            "message": event.get("message", ""),
            "object_url": event.get("object_url", ""),
            "query_url": event.get("query_url", "")
        }
        BLOCKS.append(EVENT_BLOCK)

    return json.dumps({
        "summary": ALL_SEVERITY_COUNTERS,
        "recent_events": BLOCKS
    },
    ensure_ascii=False)
$$;

In [ ]:
--- testing our 2 UDFs inside the system function to send notifications

call SYSTEM$SEND_SNOWFLAKE_NOTIFICATION(                                    -- send notification (system function)
        SNOWFLAKE.NOTIFICATION.APPLICATION_JSON(                            -- in json format (system function)
            SNOWTRAIL_DEMO.OBSERV.SNS_MESSAGE_FROM_JSON(                    -- json formatted as SNS message (UDF)
                SNOWTRAIL_DEMO.OBSERV.GET_NEW_EVENTS_AS_JSON(               -- get all new event logs as a json string (UDF)
                    timeadd(hour, -1, current_timestamp)        
                )
            )
        ),
        SNOWFLAKE.NOTIFICATION.INTEGRATION('SNOWTRAIL_DEMO_SNS_TOPIC')  -- using this notification integration
    )
;

In [ ]:
create or replace alert SNOWTRAIL_DEMO.OBSERV.NEW_ERRORS
-- no warehouse -> serverless
-- no schedule -> triggered by new data
comment = 'Streaming Alert on Event Table to publish to SNS topic'
if(exists(
    select 
        * 
    from 
        SNOWTRAIL_DEMO.OBSERV.EVENTS 
    where
        RECORD_TYPE in ('EVENT', 'LOG')
        and upper(RESOURCE_ATTRIBUTES:"snow.schema.name") = 'PIPELINE'              -- optional
        -- and upper(RESOURCE_ATTRIBUTES:"snow.database.name") = 'SNOWTRAIL_DEMO'   -- not needed as scope is this database only
        ))
then
    begin
        let FIRST_NEW_TIMESTAMP timestamp :=(
            select min(TIMESTAMP) from table(RESULT_SCAN(SNOWFLAKE.ALERT.GET_CONDITION_QUERY_UUID()))      -- get query ID from condition query above
        );

        call SYSTEM$SEND_SNOWFLAKE_NOTIFICATION(                                -- send notification (system function)
            SNOWFLAKE.NOTIFICATION.APPLICATION_JSON(                            -- in json format (system function)
                SNOWTRAIL_DEMO.OBSERV.SNS_MESSAGE_FROM_JSON(                    -- json formatted as SNS message (UDF)                 
                    SNOWTRAIL_DEMO.OBSERV.GET_NEW_EVENTS_AS_JSON(               -- get all new event logs as a json string (UDF)
                        :FIRST_NEW_TIMESTAMP
                    )
                )
            ),
            SNOWFLAKE.NOTIFICATION.INTEGRATION('SNOWTRAIL_DEMO_SNS_TOPIC')      -- using this notification integration
        );
    end;
;

### Option D) E-mail message

In [ ]:
create or replace function SNOWTRAIL_DEMO.OBSERV.HTML_EMAIL_FROM_JSON("EVENT_LOGS" VARCHAR)
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'GENERATE_HTML_TABLE'
as
$$
import json
import html

def GENERATE_HTML_TABLE(EVENT_LOGS):

    try:
        DATA = json.loads(EVENT_LOGS)
    except Exception as e:
        return f"<p><strong>Error parsing JSON:</strong> {html.escape(str(e))}</p>"

    SEV_COUNTS = DATA.get("count_new_events", [])
    RECENT_EVENTS = DATA.get("recent_events", [])

    TOTAL_EVENTS = sum(item.get("events", 0) for item in SEV_COUNTS)
    
    SEVERITY_ORDER = ['🚨 FATAL', '⛔️ ERROR', '⚠️ WARN', 'ℹ️ INFO', '🛠️ DEBUG']
    
    # Start HTML
    HTML_STRING = f"""
        <img src="https://s26.q4cdn.com/463892824/files/doc_multimedia/HI_RES-_Snowflake_Logo_Blue_1800x550.jpg"
             alt="Snowflake logo" height="72">
             
    <h2>{TOTAL_EVENTS} new Snowflake Events logged</h2>
    
    """
    
    if SEV_COUNTS:
        PARTS = []
        for SEV in SEVERITY_ORDER:
            COUNT = next((item.get('events', 0) for item in SEV_COUNTS if item.get('severity') == SEV), 0)
            if COUNT:
                PARTS.append(f"{html.escape(SEV)}: {COUNT}")
        if PARTS:
            SEV_SUMMARY = " | ".join(PARTS)
            HTML_STRING += f"<p><strong>{SEV_SUMMARY}</strong></p>\n"
        else:
            HTML_STRING += "<p><em>No new events to summarize.</em></p>\n"

    # Recent events table
    if RECENT_EVENTS:
        headers = [
            "Local time",
            "Severity",
            "Account",
            "Schema",
            "Object type",
            "Object name",
            "Object status",
            "Message",
            "Object link",
            "Event link"
        ]
        HTML_STRING += """
        <p>Most recent Events:</p>
        <table border="1" style="border-color:#DEE3EA; border-collapse:collapse;" cellpadding="5" cellspacing="0">
          <thead>
            <tr>
        """
        for col in headers:
            HTML_STRING += f'<th style="text-align:left; width:200px;">{col}</th>'
        HTML_STRING += """
            </tr>
          </thead>
          <tbody>
        """

        for EVENT in RECENT_EVENTS[:10]:
            HTML_STRING += "<tr>"

            # Local time
            TIME = html.escape(EVENT.get("local_time",""))
            HTML_STRING += f"<td>{TIME}</td>"

            # Severity
            SEV = html.escape(EVENT.get("severity",""))
            HTML_STRING += f"<td>{SEV}</td>"

            # Account name
            ACCOUNT_NAME = html.escape(EVENT.get("account",""))
            HTML_STRING += f"<td>{ACCOUNT_NAME}</td>"

            # Schema name
            DS = EVENT.get("database",""); 
            SCHEMA = EVENT.get("schema","")
            HTML_STRING += f"<td>{html.escape(f'{DS}.{SCHEMA}')}</td>"

            # Object type
            TYPE = html.escape(EVENT.get("object_type","").capitalize())
            HTML_STRING += f"<td>{TYPE}</td>"

            # Object name
            NAME = html.escape(EVENT.get("object_name",""))
            HTML_STRING += f"<td>{NAME}</td>"

            # Object status
            STATUS = html.escape(EVENT.get("object_state",""))
            HTML_STRING += f"<td>{STATUS}</td>"

            # Message
            MESSAGE = html.escape(EVENT.get("message",""))
            HTML_STRING += f"<td><pre style='margin:0'>{MESSAGE}</pre></td>"

            # Object link
            OBJECT_URL = EVENT.get("object_url","")
            if OBJECT_URL:
                HTML_STRING += f'<td><a href="{html.escape(OBJECT_URL)}">Go to Object</a></td>'
            else:
                HTML_STRING += "<td></td>"

            # Event link
            QUERY_URL = EVENT.get("query_url","")
            if QUERY_URL:
                HTML_STRING += f'<td><a href="{html.escape(QUERY_URL)}">Go to Event</a></td>'
            else:
                HTML_STRING += "<td></td>"

            HTML_STRING += "</tr>"

        HTML_STRING += """
          </tbody>
        </table>
        """
    else:
        HTML_STRING += "<p><em>No recent event details available.</em></p>"

    return HTML_STRING
$$
;


In [ ]:
create or replace notification integration SNOWTRAIL_DEMO_EMAIL
    type = EMAIL
    enabled = TRUE
    comment = 'sending Snowflake notifications to verified user emails'
;

In [ ]:
# run this cell to show the temporary input field for your email address  

import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()

st.divider()
col1, col2 = st.columns([1,1])
MY_EMAIL = col1.text_input("Verfied user email address")

if MY_EMAIL == "":
    raise Exception("user email needed to create notification integration")

In [ ]:
call SYSTEM$SEND_SNOWFLAKE_NOTIFICATION(
        SNOWFLAKE.NOTIFICATION.TEXT_HTML( 
            SNOWTRAIL_DEMO.OBSERV.HTML_EMAIL_FROM_JSON(             -- json formatted as html for email (UDF)
                SNOWTRAIL_DEMO.OBSERV.GET_NEW_EVENTS_AS_JSON(       -- get all new event logs as a json string (UDF)
                    timeadd(hour, -1, current_timestamp)            -- ENSURE THERE ARE NEW EVENTS DURING THIS TIME
                )
            )
        ),
        SNOWFLAKE.NOTIFICATION.EMAIL_INTEGRATION_CONFIG(
            'SNOWTRAIL_DEMO_EMAIL',                                 -- email integration
            'New Snowflake Event Logs',                             -- email header
            array_construct('{{MY_EMAIL}}')                         -- validated user email addresses
        )
    );

---
## 2.4. Alert setup

leveraging the "New Data Alert" (preview) on new error logs
(documentation: https://docs.snowflake.com/en/user-guide/alerts#label-alerts-type-streaming)

* the Alert contains a "stream" on the event table which triggeres the Alert action everytime new logs are added to the event table.
* the Alert action then queries the new events, formats them as a json string, adjusts the format for the selected message destination (Slack, Teams, ...) and  sends the message via the selected notification integration.

In [ ]:
create or replace alert SNOWTRAIL_DEMO.OBSERV.NEW_ERRORS
-- no warehouse -> serverless
-- no schedule -> triggered by new data
comment = 'Streaming Alert on Event Table to notify via email'
if(exists(
    select 
        * 
    from 
        SNOWTRAIL_DEMO.OBSERV.EVENTS 
    where
        RECORD_TYPE in ('EVENT', 'LOG')
        and upper(RESOURCE_ATTRIBUTES:"snow.schema.name") = 'PIPELINE'              -- optional
        -- and upper(RESOURCE_ATTRIBUTES:"snow.database.name") = 'SNOWTRAIL_DEMO'   -- not needed as scope is this database only
        ))
then
    begin
        let FIRST_NEW_TIMESTAMP timestamp :=(
            select min(TIMESTAMP) from table(RESULT_SCAN(SNOWFLAKE.ALERT.GET_CONDITION_QUERY_UUID()))      -- get query ID from condition query above
        );

        call SYSTEM$SEND_SNOWFLAKE_NOTIFICATION(                    -- send notification (system function)
            SNOWFLAKE.NOTIFICATION.TEXT_HTML(                           -- in html format (system function)
                SNOWTRAIL_DEMO.OBSERV.HTML_EMAIL_FROM_JSON(             -- json formatted as html for email (UDF)
                    SNOWTRAIL_DEMO.OBSERV.GET_NEW_EVENTS_AS_JSON(       -- get all new event logs as a json string (UDF)
                        :FIRST_NEW_TIMESTAMP       
                    )
                )
            ),
            SNOWFLAKE.NOTIFICATION.EMAIL_INTEGRATION_CONFIG(
                'SNOWTRAIL_DEMO_EMAIL',                                 -- email integration
                'New Snowflake Event Logs',                             -- email header
                array_construct('{{MY_EMAIL}}')                         -- validated user email addresses
            )
        );
    
    end;
;

In [ ]:
-- after creating the Alert object for one of the destinations above, resume the alert here to activate it

alter alert SNOWTRAIL_DEMO.OBSERV.NEW_ERRORS resume;

In [ ]:
--checking alert history if streaming Alert was triggered 

select 
    SCHEDULED_TIME,
    COMPLETED_TIME,
    STATE,
    NAME,
    SQL_ERROR_MESSAGE
from
    table(SNOWTRAIL_DEMO.INFORMATION_SCHEMA.ALERT_HISTORY(
        SCHEDULED_TIME_RANGE_START => timeadd(hour, -24, current_timestamp),
        SCHEDULED_TIME_RANGE_END => current_timestamp,
        RESULT_LIMIT => 1000,
        ALERT_NAME => 'NEW_ERRORS'
    ))
order by
    COMPLETED_TIME desc
limit 
    100;

In [ ]:
-- if alerts are triggered but notifications are not received, we can check NOTIFICATION_HISTORY to see any errors

select
    * 
from
    table(INFORMATION_SCHEMA.NOTIFICATION_HISTORY(
            START_TIME=>timeadd('hour',-1,current_timestamp()),
            RESULT_LIMIT => 100,
            INTEGRATION_NAME => 'SNOWTRAIL_DEMO_SLACK_CHANNEL'          --- insert your notification name
          )
        )
;

In [ ]:
import streamlit as st
import pandas as pd
import altair as alt
session = get_active_session()

st.header('Serverless Alert on Event Table - Costs')

SERVERLESS_CREDITS = session.sql("""
                select
                    ALERT_NAME,
                    to_date(START_TIME) as DS,
                    sum(CREDITS_USED) as CREDITS_SPENT
                from 
                    table(SNOWTRAIL_DEMO.INFORMATION_SCHEMA.SERVERLESS_ALERT_HISTORY(
                        DATE_RANGE_START => current_date - 7
                    ))
                where
                    ALERT_NAME = 'NEW_ERRORS'
                group by 
                    ALERT_NAME,
                    DS
                """).to_pandas()

CHART = alt.Chart(SERVERLESS_CREDITS).mark_bar(size=30).encode(
        x=alt.X('DS:T', axis=alt.Axis(title= None)), 
        y=alt.Y('CREDITS_SPENT:Q', axis=alt.Axis(title='Daily Credits')), 
        ).properties(height=360, width=360)

st.altair_chart(CHART)
